In [31]:
# Imports
import pandas as pd
import os
import re

In [41]:
def get_name_energy_column_name_from_number_core(number_core):
    return "CORE" + str(number_core) + "_ENERGY (J)"

TOTAL_NUMBER_CORES = 10
TOTAL_NUMBER_REPETITIONS = 30

regex = re.compile('energy_metrics_(.*).m_(\d+).csv')
files_output = os.listdir("../output/")
energy_files = [k for k in files_output if 'energy_metrics_' in k]
energy_consumption_by_script = {}
for energy_file in energy_files:
    p = re.match(regex, energy_file)
    data_raw = pd.read_csv("../output/" + energy_file)
    for number_core in range(TOTAL_NUMBER_CORES+1):
        start_energy = data_raw[get_name_energy_column_name_from_number_core(number_core)].iloc[0]
        end_energy = data_raw[get_name_energy_column_name_from_number_core(number_core)].iloc[-1]
        energy_consumption_core = float(end_energy - start_energy)
        #print(energy_consumption_core)
        #print("------")
        if p:
            script = p.group(1)
            if script in energy_consumption_by_script:
                energy_consumption_by_script[script] += energy_consumption_core
            else:
                energy_consumption_by_script[script] = energy_consumption_core
energy_consumption_by_script

{'UCI_DataSets-monks1D_optimized_gpt4': 3284.7869720456074,
 'KPMtools-optimalMatchingTest': 3288.0921630861703,
 'KPMtools-optimalMatchingTest_optimized_gpt4': 3432.5236969000543,
 'PSO-basic-main_optimized': 3381.9277191169676,
 'private-getEnvironment_optimized': 3380.2305145270075,
 'fetch_data-fetch_region_proposals_optimized': 7320.889862060547,
 'IMA-IMA-main_optimized_gpt4': 4915.01690673799,
 'tests-runalltests': 3872.0538330077543,
 'private-getEnvironment': 3293.902496338589,
 'besa-test_matlab2besa': 3417.658309937513,
 'private-getEnvironment_optimized_gpt4': 3399.8738098146277,
 'private-generatesessionid_optimized_gpt4': 3208.974639892578,
 'adminfunc-biosigpathfirst': 3288.2227630623383,
 'IMA-main': 16214.121246336785,
 'PSO-basic-main_optimized_gpt4': 3486.1990051270695,
 'afni-Test_m3dReorder_optimized_gpt4': 3292.2132110602106,
 'emd-demo_emd': 3495.4510345453164,
 'UCI_DataSets-monks1D_optimized': 3360.7108612059383,
 'UCI_DataSets-monks1D': 3321.185791015101,
 'PS

In [43]:
avg_energy_per_script_Dict = {}
for k,v in energy_consumption_by_script.items():
    avg_energy_per_script_Dict[k] = float(v) / float(TOTAL_NUMBER_REPETITIONS)

In [44]:
avg_energy_per_script_Dict

{'UCI_DataSets-monks1D_optimized_gpt4': 109.49289906818692,
 'KPMtools-optimalMatchingTest': 109.60307210287235,
 'KPMtools-optimalMatchingTest_optimized_gpt4': 114.41745656333515,
 'PSO-basic-main_optimized': 112.73092397056558,
 'private-getEnvironment_optimized': 112.67435048423359,
 'fetch_data-fetch_region_proposals_optimized': 244.0296620686849,
 'IMA-IMA-main_optimized_gpt4': 163.83389689126633,
 'tests-runalltests': 129.06846110025847,
 'private-getEnvironment': 109.79674987795298,
 'besa-test_matlab2besa': 113.92194366458376,
 'private-getEnvironment_optimized_gpt4': 113.32912699382092,
 'private-generatesessionid_optimized_gpt4': 106.9658213297526,
 'adminfunc-biosigpathfirst': 109.60742543541127,
 'IMA-main': 540.4707082112261,
 'PSO-basic-main_optimized_gpt4': 116.20663350423565,
 'afni-Test_m3dReorder_optimized_gpt4': 109.74044036867369,
 'emd-demo_emd': 116.51503448484388,
 'UCI_DataSets-monks1D_optimized': 112.02369537353128,
 'UCI_DataSets-monks1D': 110.7061930338367,
 